# Get Your Attendance Credential


### Imports

In [ ]:
from aries_cloudcontroller import AriesAgentController
import os
from termcolor import colored

### Initialise the Agent Controller

In [ ]:
api_key = os.getenv("ACAPY_ADMIN_API_KEY")
admin_url = os.getenv("ADMIN_URL")

print(f"Initialising a controller with admin api at {admin_url} and an api key of {api_key}")
agent_controller = AriesAgentController(admin_url,api_key)

### Start a Webhook Server

In [ ]:
webhook_port = int(os.getenv("WEBHOOK_PORT"))
webhook_host = "0.0.0.0"

await agent_controller.init_webhook_server(webhook_host, webhook_port)

print(f"Listening for webhooks from agent at http://{webhook_host}:{webhook_port}")

## Register Agent Event Listeners

You can see some examples within the webhook_listeners recipe. Copy any relevant cells across and customise as needed.

In [ ]:
listeners = []

In [ ]:
def problem_report_handler(payload):
    print("Problem report received")
    print(payload)

problem_report_listener = {
    "topic": "problem_report",
    "handler": problem_report_handler
}

listeners.append(problem_report_listener)

In [ ]:
## YOUR LISTENERS HERE
def holder_handler(payload):
    connection_id = payload['connection_id']
    exchange_id = payload['credential_exchange_id']
    state = payload['state']
    role = payload['role']
    print("\n---------------------------------------------------\n")
    print("Handle Issue Credential Webhook")
    print(f"Connection ID : {connection_id}")
    print(f"Credential exchange ID : {exchange_id}")
    print("Agent Protocol Role : ", role)
    print("Protocol State : ", state )
    print("\n---------------------------------------------------\n")
    print("Handle Credential Webhook Payload")
    
    if state == "offer_received":
        print("Credential Offer Recieved")
        proposal = payload["credential_proposal_dict"]
        print("The proposal dictionary is likely how you would understand and display a credential offer in your application")
        print("\n", proposal)
        print("\n This includes the set of attributes you are being offered")
        attributes = proposal['credential_proposal']['attributes']
        print(attributes)
        ## YOUR LOGIC HERE
    elif state == "request_sent":
        print("\nA credential request object contains the commitment to the agents master secret using the nonce from the offer")
        ## YOUR LOGIC HERE
    elif state == "credential_received":
        print("Received Credential")
        ## YOUR LOGIC HERE
    elif state == "credential_acked":
        ## YOUR LOGIC HERE
        credential = payload["credential"]
        print("Credential Stored\n")
        print(credential)
        
        print("\nThe referent acts as the identifier for retrieving the raw credential from the wallet")
        # Note: You would probably save this in your application database
        credential_referent = credential["referent"]
        print("Referent", credential_referent)
    


    
holder_listener = {
    "topic": "issue_credential",
    "handler": holder_handler
}

listeners.append(holder_listener)

In [ ]:
# Receive connection messages
def connections_handler(payload):
    state = payload['state']
    connection_id = payload["connection_id"]
    their_role = payload["their_role"]
    routing_state = payload["routing_state"]
    
    print("----------------------------------------------------------")
    print("Connection Webhook Event Received")
    print("Connection ID : ", connection_id)
    print("State : ", state)
    print("Routing State : ", routing_state)
    print("Their Role : ", their_role)
    print("----------------------------------------------------------")


    
    if state == "invitation":
        # Your business logic
        print("invitation")
    elif state == "request":
        # Your business logic
        print("request")

    elif state == "response":
        # Your business logic
        print("response")
    elif state == "active":
        # Your business logic
        print(colored("Connection ID: {0} is now active.".format(connection_id), "green", attrs=["bold"]))
        

connection_listener = {
    "handler": connections_handler,
    "topic": "connections"
}

listeners.append(connection_listener)

In [ ]:
agent_controller.register_listeners(listeners)

## Establish a Connection with Issuer

This is a multi-use invitation from hosted agent running in the cloud.

It's a bit hacky but code for it can be found here - https://github.com/wip-abramson/aries-issuer-service

In [ ]:

invitation =  {'@type': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/connections/1.0/invitation', '@id': 'e123d5a6-107d-4a9f-9739-c8833a96cb7a', 'label': 'Hyperledger Global Forum', 'serviceEndpoint': 'http://176.58.124.167:1020', 'recipientKeys': ['13JDgEYdau4vqMhrb1unjJkKX2MbictDazda6i6LxSxK']}


In [ ]:
auto_accept=False
alias="Hyperledger Global Forum"

invite_response = await agent_controller.connections.receive_invitation(invitation, alias, auto_accept)
connection_id = invite_response["connection_id"]

## Propose Set of Attributes for Demo Participation Credential


Scheme can be viewed on [IndyScan here](https://indyscan.io/tx/SOVRIN_STAGINGNET/domain/216849)

attributes = ["Event Name", "Participant", "Description", "Date"]


In [ ]:
from datetime import date


schema_id='W4t2Pa4XR1qBDvwhVcn4CY:2:Aries ACA ACC Jupyter Playground Demo Participation:0.0.1'
cred_def_id='QXrB6dzq3igg6jjQgcLChn:3:CL:216849:default'

event_name = "Hyperledger Global Forum 2021"
participant=input("Enter the name you wish to be identified as :")
description=input("Add any description you wish : ")
date= date.today().isoformat()


attributes = [
    {"name": "Event Name", "value": event_name},
    {"name": "Participant", "value": participant},
    {"name": "Description", "value": description},
    {"name": "Date", "value": date}
]

print(f"Proposing Aries ACA ACC Jupyter Playground Demo Participation be Issued with following attributes: \n \n {attributes}")

In [ ]:
comment="Please can you issue me this"
auto_remove="true"
trace="false"

proposal = await agent_controller.issuer.send_proposal(connection_id, schema_id, cred_def_id, attributes, comment, auto_remove, trace)

## Request Credential From Offer

Note: Your agent will automatically respond if ACAPY_AUTO_RESPOND_CREDENTIAL_OFFER=true flag is set in .env file of agent. Default is false.

To respond to an offer you must identify the offer using the credential_exchange_id generated for it. This is available from within the issue-credential holder handler. You could add custom logic in this loop `elif state == "request":` to handle this.

However, we will fetch the credential exchange records and **assume** this agent only has one record. Customise accordingly.


In [ ]:
# Optional args
thread_id=None
state = "offer_received"
role = "prover"


records_response = await agent_controller.issuer.get_records(connection_id=connection_id,state=state)
record = records_response["results"][0]
record_id = record["credential_exchange_id"]

In [ ]:
response = await agent_controller.issuer.send_request_for_record(record_id)

## Store Received Credential

This will be done automatically if the ACAPY_AUTO_STORE_CREDENTIAL=true flag is set in the .env file for this agent. Default is false.

Again you could handle this in your holder handler function in the `elif state == "credential_received":` loop.

In [ ]:
# Optionally specify an identifier to uniquely identify this credential within your agents wallet.
# You would likely want to save this somewhere.
# If not set a random one will be generated for you
credential_id = "Demo Participant"

store_cred_response = await agent_controller.issuer.store_credential(record_id, credential_id)

## Terminate Controller

Whenever you have finished with this notebook, be sure to terminate the controller. This is especially important if your business logic runs across multiple notebooks.

In [ ]:
await agent_controller.terminate()